### Sample One day data

In [5]:
from astropy.io import fits
import os
import pandas as pd
import glob
import numpy as np
from astropy import units as u
#import multiprocessing

In [26]:
#Explore_Bintable notebook has a more detailed demostration of this funcntion
def queryCatalina(filename):
    
    # open a FITS file / bintable
    hdul = fits.open(filename)  
    data = hdul[1].data 
    
    #Create reference table for exposures.
    # Risky, hardcoded to 4 exposures. Since it's hardcode on the fit table header
    refer = {'dectection_offset':[0,1,2,3], 
            'exposure_mjd_mid':[hdul[1].header["MJDMID1"], hdul[1].header["MJDMID2"], hdul[1].header["MJDMID3"], hdul[1].header["MJDMID4"]],
            'exposure_duration':[hdul[1].header["EXPOSE1"], hdul[1].header["EXPOSE2"], hdul[1].header["EXPOSE3"], hdul[1].header["EXPOSE4"]],
             'mag_sigma':[hdul[1].header["VPHDEV_1"],hdul[1].header["VPHDEV_2"],hdul[1].header["VPHDEV_3"],hdul[1].header["VPHDEV_4"]]
           }
    refer_table = pd.DataFrame(refer)
    refer_table['exposure_mjd_start'] = refer_table['exposure_mjd_mid'] - refer_table['exposure_duration'] /(3600*24)/2
    
    #Create table for observations:
    obs = {'obs_index' : data["DETS_INDEX"],
           'dectection_offset' : (data["DETS_INDEX"] - 1)% (hdul[1].header["NEXPOSE"]), #Exposure num = 4, is not hardcode here 
            'mjd' : data["TIMESTAMP"] - 2400000.5,
            'ra' : data["RA"],
            'dec' : data["Dec"],
            'ra_sigma' : data["SIGMA"] * u.arcsec.to(u.deg),
            'dec_sigma' : data["SIGMA"] * u.arcsec.to(u.deg),
            'mag' : data["MAGNITUDE"] } #'mag_sigma' : data["DELTAMU"]
    obs_table = pd.DataFrame(obs)
    obs_table = obs_table.astype({"obs_index": str}, errors='raise') 
    obs_table[["ra", "dec"]] = obs_table[["ra", "dec"]].apply(pd.to_numeric)
    
    #JOIN two tables
    final_table = pd.merge(obs_table,refer_table, on = 'dectection_offset', how='left') #probably will change to inner in future
    #Add columns
    final_table = final_table.astype({"dectection_offset": str}, errors='raise') 
    final_table['exposure_id'] = file_name[0:-5] + "_"+ final_table['dectection_offset']
    final_table['obs_id'] = final_table['exposure_id'] +"_" + final_table['obs_index'] +"_"+ final_table['dectection_offset']
    final_table['observatory_code'] =  hdul[1].header["MPCCODE"]
    final_table['filter'] =  hdul[1].header["FILTER"] 
    #Clear up
    final_table = final_table.drop(columns=['obs_index'])
    final_table = final_table.drop(columns=['dectection_offset'])
    #Formating datatype
    final_table = final_table.astype({"exposure_id": str}, errors='raise')
    final_table = final_table.astype({"obs_id": str}, errors='raise') 
    final_table = final_table.astype({"observatory_code": str}, errors='raise') 
    final_table = final_table.astype({"filter": str}, errors='raise') 
    
    return final_table
    

In [28]:
target_path = "/epyc/projects/adam_datasets/Catalina/data/"

f = sorted(glob.glob(os.path.join("20230503/*.detf")))
dfs = []

for i in f:
    #print(i)
    df = queryCatalina(i)
    dfs.append(df)
    
obs = pd.concat(dfs,ignore_index = True)
obs = obs.sort_values('mjd')
obs.to_csv(target_path + "may.csv",index = False)

In [24]:
a = pd.read_csv("/epyc/projects/adam_datasets/Catalina/data/may.csv")

In [25]:
a

,mjd,ra,dec,ra_sigma,dec_sigma,mag,exposure_mjd_mid,exposure_duration,mag_sigma,exposure_mjd_start,exposure_id,obs_id,observatory_code,filter
0,60067.131133,9.108166,2.40686,0.001664,0.001664,13.74,60067.131133,29.999741,0.276,60067.130960,703_20230503_2B_N02033_01_0001_0,703_20230503_2B_N02033_01_0001_0_1_0,703,NONE
1,60067.137088,9.108251,2.40701,0.001706,0.001706,13.51,60067.137088,29.999837,0.302,60067.136914,703_20230503_2B_N02033_01_0001_1,703_20230503_2B_N02033_01_0001_1_2_1,703,NONE
2,60067.143045,9.108337,2.40714,0.001719,0.001719,13.52,60067.143045,29.999834,0.290,60067.142871,703_20230503_2B_N02033_01_0001_2,703_20230503_2B_N02033_01_0001_2_3_2,703,NONE
3,60067.149001,9.108420,2.40731,0.001706,0.001706,13.63,60067.149001,29.999832,0.281,60067.148827,703_20230503_2B_N02033_01_0001_3,703_20230503_2B_N02033_01_0001_3_4_3,703,NONE
4,60067.131133,9.028061,4.33213,0.000619,0.000619,17.31,60067.131133,29.999741,0.276,60067.130960,703_20230503_2B_N02033_01_0001_0,703_20230503_2B_N02033_01_0001_0_5_0,703,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11767,60067.178375,9.408969,-21.87854,0.000758,0.000758,17.59,60067.178375,29.999728,0.435,60067.178201,703_20230503_2B_N02033_01_0001_3,703_20230503_2B_N02033_01_0001_3_88_3,703,NONE
11768,60067.160163,9.622944,-21.92183,0.000233,0.000233,19.61,60067.160163,29.999832,0.405,60067.159989,703_20230503_2B_N02033_01_0001_0,703_20230503_2B_N02033_01_0001_0_89_0,703,NONE
11769,60067.166240,9.624049,-21.91989,0.000542,0.000542,17.48,60067.166240,29.999828,0.443,60067.166066,703_20230503_2B_N02033_01_0001_1,703_20230503_2B_N02033_01_0001_1_90_1,703,NONE
11770,60067.172306,9.625158,-21.91773,0.000264,0.000264,17.89,60067.172306,29.999831,0.412,60067.172133,703_20230503_2B_N02033_01_0001_2,703_20230503_2B_N02033_01_0001_2_91_2,703,NONE
